In [1]:
from sklearn.cluster import KMeans
import numpy as np
from vec_db import VecDB

In [25]:
####################################################################################################################################################
############################################################ HYPER PARAMETERS ######################################################################
####################################################################################################################################################

# vector dimension
dimension = 70 

# number of subvectors for each vector
no_of_subspaces = 14
assert dimension % no_of_subspaces == 0, "Dimension must be divisible by number of subspaces"

# number of clusters for kmeans to build the codebook
no_of_clusters = 8

# how many similar vectors to search for
k = 5

In [26]:
# Divide data to subspaces 
def get_subspaces(data, no_of_subspaces : int):
    data = np.array(data)
    if data.ndim == 1: # just 1 vector
        data = np.array(data).reshape(1, -1) # so i can use np.split() lol
    return np.array(np.split(data, no_of_subspaces, axis=1))

In [27]:
print(get_subspaces([[1,2,3,4,5,6,7,8,9,10,11,12,13,14], [1,2,3,4,5,6,7,8,9,10,11,12,13,14]], 7))

[[[ 1  2]
  [ 1  2]]

 [[ 3  4]
  [ 3  4]]

 [[ 5  6]
  [ 5  6]]

 [[ 7  8]
  [ 7  8]]

 [[ 9 10]
  [ 9 10]]

 [[11 12]
  [11 12]]

 [[13 14]
  [13 14]]]


In [28]:
# Generates the codebooks that maps each subvector to a centroid
def generate_code_books(training_data, no_of_subspaces, no_of_clusters):
    codebooks = []
    subspaces = get_subspaces(training_data, no_of_subspaces)
    for subspace in subspaces:
        # Kmeans estimator to train on each supspace
        kmeans = KMeans(n_clusters=no_of_clusters, random_state=42) ###### TODO: may change this #######
        kmeans.fit(subspace)
        # Kmeans centers are our codebook for each supspace
        codebooks.append(kmeans.cluster_centers_)
    return np.array(codebooks)

In [29]:
db = VecDB(db_size=1000)
codebook = generate_code_books(db.get_all_rows(), no_of_subspaces, no_of_clusters)
print(codebook.shape)
print(codebook)


(14, 8, 5)
[[[0.44997337 0.23269832 0.31560433 0.47591147 0.80642354]
  [0.25576827 0.38199094 0.68817246 0.194339   0.45338392]
  [0.7806767  0.7228936  0.48222357 0.22787589 0.4391714 ]
  [0.57021344 0.29064944 0.20719227 0.6307481  0.26467365]
  [0.36705002 0.75684595 0.64821076 0.69584465 0.260467  ]
  [0.5656305  0.20001897 0.7403931  0.67763793 0.29954308]
  [0.53125453 0.6497312  0.71436465 0.67211676 0.79902136]
  [0.33950537 0.7824727  0.18098754 0.52840513 0.64107096]]

 [[0.22232088 0.21563518 0.6230648  0.5331292  0.33474505]
  [0.7699231  0.73689604 0.36544937 0.6897883  0.7249875 ]
  [0.26257676 0.61358494 0.38697627 0.28121006 0.7706114 ]
  [0.6958831  0.25548187 0.30443084 0.71339434 0.256062  ]
  [0.6012192  0.59827673 0.19334742 0.25121728 0.26618803]
  [0.54997844 0.2621142  0.664062   0.5924695  0.8196309 ]
  [0.7316949  0.58469224 0.7565475  0.26722866 0.38648966]
  [0.28978676 0.7960354  0.45560753 0.7337221  0.30159292]]

 [[0.26694593 0.5889511  0.23770209 0.337

In [30]:
# Encode our database vectors (convert every vector to its centroids)
def encode_data(data, codebook, no_of_subspaces):
    subspaces = get_subspaces(data, no_of_subspaces)
    encoded_data = []
    for i in range(0, no_of_subspaces):
        encoded_supspace = []
        # for each supspace
        for subvector in subspaces[i]:
            # find nearest centroid in the codebook for this supspace
            encoded_supspace.append(np.argmin([np.linalg.norm(subvector - codeword) for codeword in codebook[i]]))
        encoded_data.append(encoded_supspace)
    return np.array(encoded_data).T

In [31]:
encoded_data = encode_data(db.get_all_rows(), codebook, 14)
print(encoded_data.shape)
print(encoded_data)

(1000, 14)
[[4 6 7 ... 2 2 6]
 [0 5 6 ... 6 4 6]
 [0 3 2 ... 5 2 0]
 ...
 [2 3 0 ... 2 3 1]
 [0 0 7 ... 4 3 5]
 [6 3 3 ... 5 4 6]]


In [32]:
# Generates the distance look-up table to reduce computational time when calculating distance between vectors in searching
# table has dimensions of (#clusters, #subspaces)
def generate_distance_table(query, codebook, no_of_subspaces, no_of_clusters):
    supspaces = get_subspaces(query, no_of_subspaces)
    distance_table = np.zeros((no_of_clusters, no_of_subspaces))
    # for each cluster (centroid) calculate squared euclidean distance between the subspace and the codebook for this supspace
    for i in range(0, no_of_subspaces):
        diff = codebook[i] - supspaces[i] # difference between supspace and its codebook
        diff = diff ** 2
        # sum differences to get the squared euclidean distance
        supspace_distance = np.array(np.sum(diff, axis=1)) # dimensions: (#clusters, )
        distance_table[:, i] = supspace_distance
    return distance_table

In [33]:
distance_table = generate_distance_table([0.08925092, 0.773956, 0.6545715, 0.43887842, 0.43301523, 0.8585979, 0.085945606, 0.697368, 0.20146948, 0.094177306, 0.52647895, 0.9756223, 0.73575234, 0.7611397, 0.71747726, 0.78606427, 0.51322657, 0.12811363, 0.8397482, 0.45038593, 0.5003519, 0.370798, 0.1825496, 0.92676497, 0.78156745, 0.6438651, 0.40241432, 0.8227616, 0.5454291, 0.44341415, 0.45045954, 0.22723871, 0.092135906, 0.55458474, 0.8878898, 0.0638172, 0.85829127, 0.8276311, 0.27675968, 0.6316644, 0.16522902, 0.7580877, 0.70052296, 0.35452592, 0.06791997, 0.970698, 0.44568747, 0.89312106, 0.677919, 0.7783835, 0.75989944, 0.19463867, 0.36390603, 0.466721, 0.49779153, 0.04380375, 0.54656947, 0.15428948, 0.7433759, 0.6830489, 0.9225278, 0.7447621, 0.36664265, 0.9675097, 0.41085035, 0.32582533, 0.90553576, 0.37045968, 0.07634318, 0.4695558], codebook, no_of_subspaces, no_of_clusters)
print(distance_table.shape)
print(distance_table)

(8, 14)
[[0.67878452 0.59505981 0.64504818 0.89481125 0.50392144 0.32806341
  0.40926604 0.99054716 0.97501892 1.18705852 0.59437548 0.68023213
  0.72089492 0.46316842]
 [0.24270807 1.17814643 0.89565408 0.16513137 0.62189285 0.45960578
  1.05891837 0.97359862 0.30914343 0.22023374 0.46513622 0.47636488
  0.66729672 0.71338093]
 [0.55494079 1.19390914 0.76376431 0.44669976 0.52579976 0.39419427
  0.2138692  0.92757811 0.52745294 0.88960739 0.46839198 0.20529314
  0.22340551 0.68220798]
 [0.7302112  0.49789198 0.27632803 0.70812285 1.09626042 0.07124777
  0.39408664 0.22501026 0.62419868 0.3451618  0.33065289 0.95866703
  0.69667893 0.64806217]
 [0.17331009 0.61482626 0.85469103 0.43238939 0.04702107 0.22704674
  0.48050084 0.43634403 0.40440703 0.78864616 0.39099174 0.5347268
  0.31353926 0.55944688]
 [0.63852748 0.80655461 0.70126415 0.62695804 0.53620534 0.70739315
  0.65794761 0.61203385 1.13758676 0.60560183 0.17407295 0.35972395
  1.15750399 0.69648638]
 [0.40273482 0.35812558 0.7

In [34]:
# Query a vector to get k closest vectors
def search(encoded_data, distance_table, k, db : VecDB):
    distances = []
    # for each encoded vector in our database
    for code in encoded_data: 
        distance = 0
        i = 0
        # for each centroid in this vector
        for centroid in code: 
            # get the distance to this centroid from the distance table and sum distances up
            distance += distance_table[centroid][i]
            i += 1
        distances.append(distance)

    # get the indices of k vectors with smallest distances with the query vector
    indices = np.argsort(distances)[:k]
    results = []
    # get those vectors from the database
    for index in indices:
        results.append(db.get_one_row(index))
    return np.array(results), indices

In [35]:
results, indices = search(encoded_data, distance_table, 5, db)
print(indices)
print(results)

[  0 773 484 401 577]
[[8.92509222e-02 7.73956001e-01 6.54571474e-01 4.38878417e-01
  4.33015227e-01 8.58597875e-01 8.59456062e-02 6.97368026e-01
  2.01469481e-01 9.41773057e-02 5.26478946e-01 9.75622296e-01
  7.35752344e-01 7.61139691e-01 7.17477262e-01 7.86064267e-01
  5.13226569e-01 1.28113627e-01 8.39748204e-01 4.50385928e-01
  5.00351906e-01 3.70797992e-01 1.82549596e-01 9.26764965e-01
  7.81567454e-01 6.43865108e-01 4.02414322e-01 8.22761595e-01
  5.45429111e-01 4.43414152e-01 4.50459540e-01 2.27238715e-01
  9.21359062e-02 5.54584742e-01 8.87889802e-01 6.38172030e-02
  8.58291268e-01 8.27631116e-01 2.76759684e-01 6.31664395e-01
  1.65229023e-01 7.58087695e-01 7.00522959e-01 3.54525924e-01
  6.79199696e-02 9.70697999e-01 4.45687473e-01 8.93121064e-01
  6.77918971e-01 7.78383493e-01 7.59899437e-01 1.94638669e-01
  3.63906026e-01 4.66720998e-01 4.97791529e-01 4.38037515e-02
  5.46569467e-01 1.54289484e-01 7.43375897e-01 6.83048904e-01
  9.22527790e-01 7.44762123e-01 3.66642654e-01 9

In [36]:
def train(training_data, no_of_clusters):
    kmeans = KMeans(n_clusters=no_of_clusters, random_state=42)
    kmeans.fit(training_data)
    assignments = kmeans.labels_
    cluster_to_ids : dict[int: list[int]]= {}
    for i in range(0, no_of_clusters):
        cluster_to_ids[i] = []
    for i in range(0, len(assignments)):
        cluster_to_ids[assignments[i]].append(i)
    return kmeans, cluster_to_ids

In [38]:
db = VecDB(db_size=1000)
kmeans, cluster_to_ids = train(db.get_all_rows(), 50)
print(kmeans.cluster_centers_.shape)
for c, ids in cluster_to_ids.items():
    print(len(ids))
print(cluster_to_ids)

(50, 70)
16
30
17
29
15
8
11
19
7
7
18
19
25
9
14
18
45
40
2
28
40
23
20
25
20
22
20
15
12
28
17
29
33
12
4
20
39
18
28
27
36
15
15
7
37
11
13
16
7
14
{0: [15, 65, 105, 137, 147, 148, 199, 374, 606, 676, 743, 756, 766, 807, 831, 909], 1: [22, 85, 95, 128, 129, 184, 208, 236, 267, 292, 294, 363, 401, 424, 455, 467, 483, 502, 597, 604, 628, 646, 739, 851, 900, 916, 927, 930, 932, 981], 2: [56, 337, 339, 406, 411, 443, 496, 518, 593, 688, 714, 754, 874, 915, 955, 986, 996], 3: [67, 71, 84, 185, 203, 273, 422, 447, 514, 569, 584, 592, 600, 601, 610, 632, 699, 727, 729, 732, 751, 832, 843, 852, 875, 918, 969, 978, 999], 4: [13, 42, 54, 167, 177, 181, 304, 487, 508, 764, 776, 779, 805, 811, 857], 5: [18, 122, 253, 331, 366, 470, 550, 724], 6: [268, 326, 419, 465, 513, 516, 546, 547, 778, 795, 879], 7: [37, 111, 142, 151, 204, 240, 287, 328, 580, 616, 668, 694, 753, 775, 783, 793, 924, 947, 948], 8: [113, 409, 428, 545, 682, 737, 854], 9: [103, 227, 454, 493, 501, 505, 897], 10: [62, 86, 110,

In [86]:
def cos_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
    return cosine_similarity

In [96]:
def search(query, estimator : KMeans, clusters_to_ids : dict[int, list[int]], db : VecDB, k : int):
    kmeans.cluster_centers_ = kmeans.cluster_centers_.astype(float)
    query = np.array(query).reshape((1,-1))
    closest_clusters = estimator.predict(query)
    distances = []
    for closest_cluster in closest_clusters:
        ids = clusters_to_ids[closest_cluster]
        for id in ids:
            vector = db.get_one_row(id)
            cos = cos_similarity(query,vector)
            distances.append((cos, id))
    distances = sorted(distances, key= lambda x : x[0], reverse=True)
    return [d[1] for d in distances[:k]]

In [97]:
indices = search([0.08925092, 0.773956, 0.6545715, 0.43887842, 0.43301523, 0.8585979, 0.085945606, 0.697368, 0.20146948, 0.094177306, 0.52647895, 0.9756223, 0.73575234, 0.7611397, 0.71747726, 0.78606427, 0.51322657, 0.12811363, 0.8397482, 0.45038593, 0.5003519, 0.370798, 0.1825496, 0.92676497, 0.78156745, 0.6438651, 0.40241432, 0.8227616, 0.5454291, 0.44341415, 0.45045954, 0.22723871, 0.092135906, 0.55458474, 0.8878898, 0.0638172, 0.85829127, 0.8276311, 0.27675968, 0.6316644, 0.16522902, 0.7580877, 0.70052296, 0.35452592, 0.06791997, 0.970698, 0.44568747, 0.89312106, 0.677919, 0.7783835, 0.75989944, 0.19463867, 0.36390603, 0.466721, 0.49779153, 0.04380375, 0.54656947, 0.15428948, 0.7433759, 0.6830489, 0.9225278, 0.7447621, 0.36664265, 0.9675097, 0.41085035, 0.32582533, 0.90553576, 0.37045968, 0.07634318, 0.4695558], kmeans, cluster_to_ids, db, 5)
print(indices)


[0, 484, 423, 869, 396]
